# Task 4 — Optimize Portfolio Based on Forecast (MPT)

Objective: Use the TSLA forecast (Task 3) + historical SPY/BND returns to build an optimized portfolio using Modern Portfolio Theory.

## Rubric mapping
- **(a) Expected Returns**: TSLA from forecast, SPY/BND from historical mean (annualized)
- **(b) Covariance Matrix**: computed from historical daily returns for TSLA/SPY/BND
- **(c) Efficient Frontier**: generated via `scipy.optimize` and visualized
- **(d) Key Portfolios**: Max Sharpe and Min Vol identified and marked
- **Recommendation**: final portfolio weights + expected return + volatility + Sharpe + 1 paragraph justification


In [ ]:
# 0) Setup: imports + repo-root detection
import os, sys, subprocess
from pathlib import Path
import pandas as pd

def find_repo_root(start: Path | None = None) -> Path:
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / 'src').exists() and (cur / 'data').exists():
            return cur
        cur = cur.parent
    raise RuntimeError('Could not auto-detect repo root. Run from inside the repo.')

REPO_ROOT = find_repo_root(Path.cwd())
print('Notebook working directory:', os.getcwd())
print('Repo root:', REPO_ROOT)

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))


## 1) Verify required artifacts exist

- `data/task1/processed/returns.parquet`
- `outputs/task3/forecasts/tsla_logret_forecast_6m.csv` (or 12m)


In [ ]:
from src import task4_config as cfg

required = [
    REPO_ROOT / cfg.RETURNS_PATH,
    REPO_ROOT / (cfg.TSLA_FORECAST_LOGRET_6M if cfg.TSLA_FORECAST_HORIZON == '6m' else cfg.TSLA_FORECAST_LOGRET_12M)
]

missing = [p for p in required if not p.exists()]
for p in required:
    print('OK' if p.exists() else 'MISSING', p)

if missing:
    raise FileNotFoundError('Missing required files:\n' + '\n'.join(map(str, missing)))


## 2) Run Task 4 optimization script (creates all deliverables)

This generates:
- Efficient frontier plot (with Max Sharpe + Min Vol marked)
- Covariance heatmap
- Portfolio weights + metrics
- 1-paragraph recommendation


In [ ]:
def run(cmd):
    print('Running:', ' '.join(cmd))
    subprocess.check_call(cmd, cwd=str(REPO_ROOT))

runner = REPO_ROOT / 'scripts' / 'task4_run_all.py'
if runner.exists():
    run([sys.executable, 'scripts/task4_run_all.py'])
else:
    run([sys.executable, 'scripts/task4_optimize_portfolio.py'])

print('Task 4 complete.')


## 3) Deliverables: figures (rubric b & c & d)

- Covariance matrix heatmap
- Efficient frontier with Min Vol and Max Sharpe portfolios marked


In [ ]:
from IPython.display import Image, display

fig_dir = REPO_ROOT / 'outputs' / 'task4' / 'figures'
heatmap = fig_dir / 'covariance_heatmap.png'
frontier = fig_dir / 'efficient_frontier.png'

for p in [heatmap, frontier]:
    print('OK' if p.exists() else 'MISSING', p)
    if p.exists():
        display(Image(filename=str(p)))


## 4) Deliverables: tables + recommendation (rubric a & recommendation)

We display:
- Expected returns (annual)
- Covariance matrix (annual)
- Portfolio weights
- Final recommendation paragraph


In [ ]:
tbl_dir = REPO_ROOT / 'outputs' / 'task4' / 'tables'
sum_dir = REPO_ROOT / 'outputs' / 'task4' / 'summaries'

exp_ret = tbl_dir / 'expected_returns_annual.csv'
cov = tbl_dir / 'covariance_annual.csv'
wts = tbl_dir / 'portfolio_weights.csv'
rec = sum_dir / 'portfolio_recommendation.md'

for p in [exp_ret, cov, wts, rec]:
    print('OK' if p.exists() else 'MISSING', p)

if exp_ret.exists():
    display(pd.read_csv(exp_ret))

if cov.exists():
    display(pd.read_csv(cov).head())

if wts.exists():
    display(pd.read_csv(wts))

if rec.exists():
    print(rec.read_text(encoding='utf-8'))


## 5) Deliverables checklist (submission)

**Figures**
- `outputs/task4/figures/covariance_heatmap.png`
- `outputs/task4/figures/efficient_frontier.png`

**Tables**
- `outputs/task4/tables/expected_returns_annual.csv`
- `outputs/task4/tables/covariance_annual.csv`
- `outputs/task4/tables/portfolio_weights.csv`

**Recommendation + metrics**
- `outputs/task4/summaries/portfolio_recommendation.md`
- `outputs/task4/task4_summary.json`
